In [3]:
import csv
import numpy as np
import networkx as nx
import math
from scipy.stats import mode
import function
import pickle

# Part5

## num_user_to_ratio,num_user_to_geofly_ratio

In [ ]:
def get_node_use_as_end_rate(path):

    i = 0
    user_count = {}
    csv_reader = csv.reader(open('./data/temp/user_count.csv', encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        user_count[row[0]] = eval(row[1])
    
    
    end_node_user_dic = {}# 记录那些用户使用了终点
    user_end_dic = {}
    
    i = 0
    csv_reader = csv.reader(open(path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        
        user = row[1]
        end = row[6]
        
        # 看每个终点用了几次
        if user not in user_end_dic:
            user_end_dic[user] = {}
            user_end_dic[user][end] = 1
        else:
            if end not in user_end_dic[user]:
                user_end_dic[user][end] = 1
            else:
                user_end_dic[user][end] += 1
        
        # 记录那些用户在那个终点到达过
        if end not in end_node_user_dic:
            end_node_user_dic[end] = set()
            end_node_user_dic[end].add(user)
        else:
            end_node_user_dic[end].add(user)

    
    node_use_as_end_rate = {}
    
    for node in end_node_user_dic:
        total_to = 0
        total_used = 0
        for user in end_node_user_dic[node]:
            total_to+=user_end_dic[user][node]
            total_used+=user_count[user]
        if total_used!=0:
            node_use_as_end_rate[node] = total_to/total_used
        else:
            node_use_as_end_rate[node] = 0
            
    return node_use_as_end_rate

In [ ]:
num_user_to_ratio = get_node_use_as_end_rate('./data/train_hard.csv')
num_user_to_geofly_ratio = get_node_use_as_end_rate('./data/train_end_geofly.csv')

In [ ]:
with open('./data/temp/feature_table/num_user_to_ratio','wb+') as f:
    pickle.dump(num_user_to_ratio, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/feature_table/num_user_to_geofly_ratio','wb+') as f:
    pickle.dump(num_user_to_geofly_ratio, f, pickle.HIGHEST_PROTOCOL)


## num_user_from_ratio,num_user_from_geofly_ratio


In [ ]:
def get_node_use_as_start_rate(path):

    i = 0
    user_count = {}
    csv_reader = csv.reader(open('./data/temp/user_count.csv', encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        user_count[row[0]] = eval(row[1])
    
    
    start_node_user_dic = {}# 记录那些用户从点出发
    user_start_dic = {}
    
    i = 0
    csv_reader = csv.reader(open(path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        
        user = row[1]
        start = row[5]
        
        # 看每个起点用了几次
        if user not in user_start_dic:
            user_start_dic[user] = {}
            user_start_dic[user][start] = 1
        else:
            if start not in user_start_dic[user]:
                user_start_dic[user][start] = 1
            else:
                user_start_dic[user][start] += 1
        
        # 记录那些用户在那个终点到达过
        if start not in start_node_user_dic:
            start_node_user_dic[start] = set()
            start_node_user_dic[start].add(user)
        else:
            start_node_user_dic[start].add(user)

    
    node_use_as_start_rate = {}
    
    for node in start_node_user_dic:
        total_from = 0
        total_used = 0
        for user in start_node_user_dic[node]:
            total_from+=user_start_dic[user][node]
            total_used+=user_count[user]
        if total_used!=0:
            node_use_as_start_rate[node] = total_from/total_used
        else:
            node_use_as_start_rate[node] = 0
            
    return node_use_as_start_rate

In [ ]:
num_user_from_ratio = get_node_use_as_start_rate('./data/train_hard.csv')
num_user_from_geofly_ratio = get_node_use_as_start_rate('./data/train_start_geofly.csv')

In [ ]:
with open('./data/temp/feature_table/num_user_from_ratio','wb+') as f:
    pickle.dump(num_user_from_ratio, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/feature_table/num_user_from_geofly_ratio','wb+') as f:
    pickle.dump(num_user_from_geofly_ratio, f, pickle.HIGHEST_PROTOCOL)

# Part6

# start_to_end_ratio,start_geofly_to_end_ratio
# start_to_end_geofly_ratio,start_geofly_to_end_geofly_ratio

In [14]:
def get_start_to_end_ratio(start_to_end_count_path,start_count_path):
    # 一个是起点到终点次数计算的csv的路径，一个是计算start出发了几次的csv路径，
    # 在起点飘逸的时候需要给start_count_path也是飘逸的
    
    node_start_count = {}# 节点出发次数
    i = 0
    csv_reader = csv.reader(open(start_count_path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        start = row[5]
        if start not in node_start_count:
            node_start_count[start] = 1
        else:
            node_start_count[start] += 1
    
    node_start_to_end_count = {}# 起点到终点的次数
    
    i = 0
    csv_reader = csv.reader(open(start_to_end_count_path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        
        user = row[1]
        start = row[5]
        end = row[6]
        
        if start not in node_start_to_end_count:
            node_start_to_end_count[start] = {}
            node_start_to_end_count[start][end] = 1
        else:
            if end not in node_start_to_end_count[start]:
                node_start_to_end_count[start][end] = 1
            else:
                node_start_to_end_count[start][end] += 1
        
    node_start_to_end_count_ratio = {}
    
    for start in node_start_to_end_count:
        node_start_to_end_count_ratio[start] = {}
        for end in node_start_to_end_count[start]:
            node_start_to_end_count_ratio[start][end] = node_start_to_end_count[start][end]/\
                                                        node_start_count[start]
    
    return node_start_to_end_count_ratio

In [17]:
node_start_to_end_count_ratio = get_start_to_end_ratio('./data/train_hard.csv',\
                                                       './data/train_hard.csv')
node_start_geofly_to_end_count_ratio = \
                                    get_start_to_end_ratio('./data/train_start_geofly.csv',\
                                                           './data/train_start_geofly.csv')

node_start_to_end_geofly_count_ratio = \
                                    get_start_to_end_ratio('./data/train_end_geofly.csv',\
                                                           './data/train_hard.csv')


In [22]:
node_start_geofly_to_end_geofly_count_ratio = \
                                    get_start_to_end_ratio('./data/train_start_geofly_end_geofly.csv',\
                                                           './data/train_start_geofly.csv')

In [46]:
with open('./data/temp/feature_table/node_start_to_end_count_ratio','wb+') as f:
    pickle.dump(node_start_to_end_count_ratio, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/feature_table/node_start_geofly_to_end_count_ratio','wb+') as f:
    pickle.dump(node_start_geofly_to_end_count_ratio, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/feature_table/node_start_to_end_geofly_count_ratio','wb+') as f:
    pickle.dump(node_start_to_end_geofly_count_ratio, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/feature_table/node_start_geofly_to_end_geofly_count_ratio','wb+') as f:
    pickle.dump(node_start_geofly_to_end_geofly_count_ratio, f, pickle.HIGHEST_PROTOCOL)

# Part7

In [66]:
def get_end_to_end_ratio(start_to_end_count_path,end_count_path):
    # 一个是起点到终点次数计算的csv的路径，一个是计算end 到达了几次的csv路径，
    # 在起点飘逸的时候需要给end_count_path也是飘逸的
    
    node_end_count = {}# 节点到达次数
    i = 0
    csv_reader = csv.reader(open(end_count_path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        end = row[6]
        if end not in node_end_count:
            node_end_count[end] = 1
        else:
            node_end_count[end] += 1
    
    node_start_to_end_count = {}# 起点到终点的次数
    
    i = 0
    csv_reader = csv.reader(open(start_to_end_count_path, encoding='utf-8'))
    for row in csv_reader:
        i+=1
        if i == 1:
            continue
        
        user = row[1]
        start = row[5]
        end = row[6]
        
        if start not in node_start_to_end_count:
            node_start_to_end_count[start] = {}
            node_start_to_end_count[start][end] = 1
        else:
            if end not in node_start_to_end_count[start]:
                node_start_to_end_count[start][end] = 1
            else:
                node_start_to_end_count[start][end] += 1
        
    node_start_to_end_count_ratio = {}
    
    
    for start in node_start_to_end_count:
        node_start_to_end_count_ratio[start] = {}
        for end in node_start_to_end_count[start]:
            node_start_to_end_count_ratio[start][end] = node_start_to_end_count[start][end]/\
                                                        node_end_count[end]
    
    return node_start_to_end_count_ratio

In [54]:
node_start_to_end_count_ratio = get_end_to_end_ratio('./data/train_hard.csv',\
                                                       './data/train_hard.csv')

In [67]:
node_start_to_end_count_ratio = get_end_to_end_ratio('./data/train_start_geofly.csv',\
                                                       './data/train_hard.csv')

In [72]:
for start in node_start_geofly_to_end_geofly_count_ratio:
    t = 0
    for end in node_start_geofly_to_end_geofly_count_ratio[start]:
        if node_start_geofly_to_end_geofly_count_ratio[start][end] > 1:
            print(start)

In [70]:
node_start_to_end_count_ratio = get_end_to_end_ratio('./data/train_end_geofly.csv',\
                                                       './data/train_end_geofly.csv')

In [ ]:
node_start_to_end_count_ratio = get_end_to_end_ratio('./data/train_end_geofly.csv',\
                                                       './data/train_end_geofly.csv')